In [1]:
import sys
sys.path.append('..')
import selex_dca, utils

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import utils
import selex_dca
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

from matplotlib import cm
import pickle

/home/scrotti/Aptamer2025py/experiments/../selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
experiment_id_idx = 1
experiment_ids = ['Dop8V030', 'Dop8V930', 'Dop8V2430']
experiment_id = experiment_ids[experiment_id_idx]
round_ids = ["ARN", "R01", "R02N"]

In [3]:
dtype = torch.float32
device = utils.best_device()

Selected device: cuda


In [11]:
sequences = utils.sequences_from_files(experiment_id, round_ids)

Extracting sequences...
Finished round ARN
Finished round R01
Finished round R02N


In [12]:
def hamming(x, y):
    return (x != y).sum(dim=(-1))

In [13]:
with open('./saved/wildtypes_sites.pkl', 'rb') as f:
    wts = pickle.load(f)

wt = wts[experiment_id_idx]
wt_oh = utils.one_hot(wt)

In [23]:
L = len(sequences[0][0])

In [14]:
distances = [hamming(wt, seq) for seq in sequences]

In [19]:
d_sum = torch.tensor([d.sum().item() for d in distances])
total_reads = torch.tensor([s.size(0) for s in sequences])

In [29]:
ts = torch.arange(len(round_ids))
A = (ts * d_sum).sum()
B = (ts * total_reads).sum()
C = d_sum.sum()
D = total_reads.sum()

In [28]:
beta = - torch.log(L*B/A-1)
beta

tensor(-1.4322)

In [32]:
D/C, total_reads[0]/d_sum[0]

(tensor(0.0887), tensor(0.0787))

In [30]:
gamma = - torch.log(L*D/C-1)
gamma

tensor(-1.0960)

In [31]:
mu = 0.3
q = 4
from math import log
log(mu/(q-1)) - log(1-mu)

-1.9459101490553135